In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import re

import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    col,
    desc,
    explode,
    row_number,
    sum as _sum,
    udf,
    year
)

In [3]:
findspark.init()

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [4]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("LR2")\
        .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.14.0")\
        .getOrCreate()

In [5]:
programming_languages = spark.read\
        .option('header', 'true')\
        .option('inferSchema', 'true')\
        .csv('/content/programming-languages.csv')

In [6]:
programming_languages.show(10, truncate=False)

+----------+---------------------------------------------------------+
|name      |wikipedia_url                                            |
+----------+---------------------------------------------------------+
|A# .NET   |https://en.wikipedia.org/wiki/A_Sharp_(.NET)             |
|A# (Axiom)|https://en.wikipedia.org/wiki/A_Sharp_(Axiom)            |
|A-0 System|https://en.wikipedia.org/wiki/A-0_System                 |
|A+        |https://en.wikipedia.org/wiki/A%2B_(programming_language)|
|A++       |https://en.wikipedia.org/wiki/A%2B%2B                    |
|ABAP      |https://en.wikipedia.org/wiki/ABAP                       |
|ABC       |https://en.wikipedia.org/wiki/ABC_(programming_language) |
|ABC ALGOL |https://en.wikipedia.org/wiki/ABC_ALGOL                  |
|ABSET     |https://en.wikipedia.org/wiki/ABSET                      |
|ABSYS     |https://en.wikipedia.org/wiki/ABSYS                      |
+----------+---------------------------------------------------------+
only s

In [7]:
posts_data = spark.read \
    .format("xml") \
    .option("rowTag", "row") \
    .load("/content/posts_sample.xml")

In [8]:
posts_data.show(10)

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+----------

In [9]:
known_languages = set([language.name.lower() for language in programming_languages.select('name').collect()])


In [10]:
list(known_languages)[:20]

['netlogo',
 'ttcn',
 'supertalk',
 'gap',
 'net.data',
 'newtonscript',
 'tom',
 'c++ – iso/iec 14882',
 'turing',
 'pl/i – iso 6160',
 'imp',
 'x10',
 'joss',
 'mathematica',
 'nsis',
 'promal',
 'emerald',
 'lyapas',
 'prograph',
 'cel']

In [11]:
def extract_languages(tag_string: str) -> list:
  if not tag_string:
    return []
  found = re.findall(r'<([^<>]+)>', tag_string)
  return [lang for lang in found if lang in known_languages]

In [12]:
extract_langs_udf = udf(extract_languages, ArrayType(StringType()))
posts_data_langs = (
    posts_data
    # Cтолбец-массив языков
    .withColumn('languages', extract_langs_udf(posts_data['_Tags']))
    # Год создания
    .withColumn('year', year(posts_data['_CreationDate']))
    # Разворачиваем массив в строки
    .withColumn('lang', explode(col('languages')))
    # Оставляем только нужные колонки
    .select('year', 'lang',"_ViewCount")
    # Фильтруем по интервалу годов
    .filter(col("year").between(2010, 2020))
)


In [13]:
posts_data_langs.show(10, truncate=False)

+----+-----------+----------+
|year|lang       |_ViewCount|
+----+-----------+----------+
|2010|java       |132       |
|2010|php        |1258      |
|2010|ruby       |9649      |
|2010|c          |2384      |
|2010|php        |1987      |
|2010|python     |3321      |
|2010|javascript |128       |
|2010|applescript|477       |
|2010|php        |1748      |
|2010|php        |998       |
+----+-----------+----------+
only showing top 10 rows



In [14]:
year_window = Window.partitionBy("year").orderBy(desc("popularity"))

In [15]:
top_languages_per_year = (
    posts_data_langs
    # Группировка и суммирование просмотров
    .groupBy("year", "lang")
    .agg(_sum(col("_ViewCount")).alias("popularity"))
    # Присвоение ранга внутри каждого года
    .withColumn("rank", row_number().over(year_window))
    # Оставляем только топ-10
    .filter(col("rank").between(1, 10))
)

In [16]:
top_languages_per_year.show(10)

+----+-----------+----------+----+
|year|       lang|popularity|rank|
+----+-----------+----------+----+
|2010|        php|   1189629|   1|
|2010|       java|    563211|   2|
|2010| javascript|    316131|   3|
|2010|objective-c|     97009|   4|
|2010|       ruby|     76215|   5|
|2010|          c|     66587|   6|
|2010|     python|     60672|   7|
|2010|     matlab|     51865|   8|
|2010|applescript|     32305|   9|
|2010|     delphi|     13065|  10|
+----+-----------+----------+----+
only showing top 10 rows



In [17]:
# Сохраняем результат в Parquet
top_languages_per_year.write \
    .mode("overwrite") \
    .parquet("top_languages_per_year.parquet")

In [18]:
posts_data \
  .withColumn("year", year(col("_CreationDate"))) \
  .select("year") \
  .distinct() \
  .orderBy("year") \
  .show()


+----+
|year|
+----+
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
+----+



In [19]:
(
    spark.read
         .parquet("top_languages_per_year.parquet")
         .orderBy("year", "rank")
         .show(110, truncate=False)
)

+----+-----------+----------+----+
|year|lang       |popularity|rank|
+----+-----------+----------+----+
|2010|php        |1189629   |1   |
|2010|java       |563211    |2   |
|2010|javascript |316131    |3   |
|2010|objective-c|97009     |4   |
|2010|ruby       |76215     |5   |
|2010|c          |66587     |6   |
|2010|python     |60672     |7   |
|2010|matlab     |51865     |8   |
|2010|applescript|32305     |9   |
|2010|delphi     |13065     |10  |
|2011|javascript |809078    |1   |
|2011|java       |389834    |2   |
|2011|php        |246770    |3   |
|2011|c          |238277    |4   |
|2011|objective-c|218934    |5   |
|2011|python     |203180    |6   |
|2011|bash       |60805     |7   |
|2011|ruby       |39223     |8   |
|2011|perl       |28502     |9   |
|2011|matlab     |18816     |10  |
|2012|java       |661770    |1   |
|2012|javascript |572750    |2   |
|2012|php        |459506    |3   |
|2012|python     |275337    |4   |
|2012|ruby       |104610    |5   |
|2012|objective-c|94